In [ ]:
from lec_utils import *
# For illustration purposes only.
def err():
    raise ValueError('😭😭😭 I just deleted all of your files! 😭😭😭')

<div class="alert alert-info" markdown="1">

#### Lecture 10

# APIs, SQL, and Spreadsheets

### EECS 398-003: Practical Data Science, Fall 2024

<small><a style="text-decoration: none" href="https://practicaldsc.org">practicaldsc.org</a> • <a style="text-decoration: none" href="https://github.com/practicaldsc/fa24">github.com/practicaldsc/fa24</a></small>
    
</div>

### Announcements 📣

- Homework 4 is due **tonight**. Watch [**this video**](https://www.loom.com/share/58287a89121545fdbd0131d22c2d9c94?sid=d1aff353-d54c-4d9e-b55b-7b5015f683a8) before submitting!<br><small>TL;DR: Make sure `hw04.ipynb` is smaller than 1 MB before submitting.</small>
- The Midterm Exam is in a few weeks – start working through old exam problems [**here**](https://study.practicaldsc.org/).<br><small>You can even see an example old exam PDF.</small>
- Looking for sources of data, or other supplemental resources? Look at our updated [**Resources**](https://practicaldsc.org/resources) page!

### Agenda

- Example: Scraping the Happening @ Michigan page.
- APIs and JSON.
- Generalized table manipulation.
    - SQL.
    - Google Sheets.

<div class="alert alert-success" markdown="1">
    <h3>Activity</h3>
    
Consider the following HTML document, which represents a webpage containing the top few songs with the most streams on Spotify today in Canada.

```html
<head>
    <title>3*Canada-2022-06-04</title>
</head>
<body>
    <h1>Spotify Top 3 - Canada</h1>
    <table>
        <tr class='heading'>
            <th>Rank</th>
            <th>Artist(s)</th> 
            <th>Song</th>
        </tr>
        <tr class=1>
            <td>1</td>
            <td>Harry Styles</td> 
            <td>As It Was</td>
        </tr>
        <tr class=2>
            <td>2</td>
            <td>Jack Harlow</td> 
            <td>First Class</td>
        </tr>
        <tr class=3>
            <td>3</td>
            <td>Kendrick Lamar</td> 
            <td>N95</td>
        </tr>
    </table>
</body>
```

- **Part 1**: How many leaf nodes are there in the DOM tree of the previous document — that is, how many nodes have no children?

- **Part 2**: What does the following line of code evaluate to?

```python
        len(soup.find_all("td"))
```

- **Part 3**: What does the following line of code evaluate to?

```python
        soup.find("tr").get("class")
```

## Example: Scraping the Happening @ Michigan page

---

### Example: Scraping the Happening @ Michigan page

- Our goal is to create a DataFrame with the information about each event at [events.umich.edu](https://events.umich.edu).

<center><img src="imgs/events.png" width=800><small>Today's event page will look different, since I wrote this part of the lecture a few days ago!</small></center>

In [ ]:
import requests
from bs4 import BeautifulSoup

In [ ]:
res = requests.get('https://events.umich.edu') 
res

In [ ]:
soup = BeautifulSoup(res.text)

- Let's start by opening the page in Chrome, right clicking on the page, and clicking "Inspect".<br><small>As we can see, the HTML is much more complicated this time – this is usually the case for real websites!</small>

### Identifying `<div>`s

- It's not easy identifying which `<div>`s we want. The Inspect tool makes this easier, but it's good to verify that `find_all` is finding the right number of elements.

In [ ]:
divs = soup.find_all(class_='col-xs-12') 

In [ ]:
len(divs)

- Again, let's deal with one `<div>` at a time. First, we should extract the title of the event.

In [ ]:
divs[0]

In [ ]:
divs[0].find('div', class_='event-title').find('a').get('title') 

- The time and location, too.

In [ ]:
divs[0].find('time').get('datetime') 

In [ ]:
divs[0].find('ul').find('a').get('title') 

### Parsing a single event, and then every event

- As before, we'll implement a function that takes in a BeautifulSoup object corresponding to a single `<div>` and returns a dictionary with the relevant information about that event.

In [ ]:
def process_event(div):
    title = div.find('div', class_='event-title').find('a').get('title')
    location = div.find('ul').find('a').get('title')
    time = pd.to_datetime(div.find('time').get('datetime')) # Good idea!
    return {'title': title, 'time': time, 'location': location}

In [ ]:
process_event(divs[12])

- Now, we can call it on every `<div>` in `divs`.<br><small>Remember, we already ran `divs = soup.find_all(class_='col-xs-12')`.</small>

In [ ]:
row_list = []
for div in divs:
    try:
        row_list.append(process_event(div))
    except Exception as e:
        print(e)

In [ ]:
events = pd.DataFrame(row_list) 
events.head()

- Now, `events` is a DataFrame, like any other!

In [ ]:
# Which events are in-person today?
events[~events['location'].isin(['Virtual', ''])] 

### Web data in practice

- [The spread of true and false news online](https://www.science.org/doi/full/10.1126/science.aap9559) by Vosoughi et al. compared how true and false news spreads via X (Twitter):

> There is worldwide concern over false news and the possibility that it can influence political, economic, and social well-being. To understand how false news spreads, Vosoughi et al. used a data set of rumor cascades on Twitter from 2006 to 2017. About 126,000 rumors were spread by ∼3 million people. False news reached more people than the truth; the top 1% of false news cascades diffused to between 1000 and 100,000 people, whereas the truth rarely diffused to more than 1000 people. Falsehood also diffused faster than the truth. The degree of novelty and the emotional reactions of recipients may be responsible for the differences observed.

- To conduct this study, the authors used the X API for accessing tweets and web-scraped fact-checking websites to verify whether news was false or not.

## APIs and JSON

---

### Recap: Scraping vs. APIs

- There are two ways to programmatically access data on the internet: either **by scraping**, or **through an API**.

- **Scraping** is the act of emulating a web browser to access its HTML source code.<br><small>When scraping, you get back data as HTML and have to **parse** that HTML to extract the information you want. Parse means to "extract meaning from a sequence of symbols".

- An application programming interface, or **API**, is a service that makes data directly available to the user in a **convenient** fashion. Usually, APIs give us code back as JSON objects.<br><small>APIs are made by organizations that host data. For example, X (formally known as Twitter) has an [API](https://developer.twitter.com/en/docs/twitter-api), as does [OpenAI](https://platform.openai.com/docs/overview?lang=python), the creators of ChatGPT.</small>

- To understand how to use an API, we must learn how to work with JSON objects.

<center><img src='imgs/json.png' width=50%></center>

### JSON

- JSON stands for **JavaScript Object Notation**. It is a lightweight format for storing and transferring data.

- It is:
    - very easy for computers to read and write.
    - moderately easy for programmers to read and write by hand.
    - meant to be generated and parsed.

- Most modern languages have an interface for working with JSON objects.<br><small>JSON objects _resemble_ Python dictionaries (but are not the same!).</small>

### JSON data types

| Type | Description |
| --- | --- |
| String | Anything inside double quotes. |
| Number | Any number (no difference between ints and floats). |
| Boolean | `true` and `false`. |
| Null | JSON's empty value, denoted by `null`. |
| Array | Like Python lists. |
| Object | A collection of key-value pairs, like dictionaries. Keys must be strings, values can be anything (even other objects). |

See [json-schema.org](https://json-schema.org/understanding-json-schema/reference/type.html) for more details.

### Example JSON object

See `data/family.json`.

<center><img src='imgs/hierarchy.png' width=50%></center>

In [ ]:
!cat data/family.json

In [ ]:
import json
with open('data/family.json', 'r') as f:
    family_str = f.read()
    family_tree = json.loads(family_str)

In [ ]:
family_tree

In [ ]:
family_tree['children'][1]['children'][0]['age'] 

<div class="alert alert-danger" markdown="1">

#### Reference Slide

### Aside: `eval`

- `eval`, which stands for "evaluate", is a function built into Python.

- It takes in a **string containing a Python expression** and evaluates it in the current context.

In [ ]:
x = 4
eval('x + 5')

- It seems like `eval` can do the same thing that `json.loads` does...

In [ ]:
eval(family_str)

- But you should **almost never use `eval`**.

In [ ]:
with open('data/evil_family.json', 'r') as f:
    evil_family_str = f.read()
eval(evil_family_str)

- Oh no! Since `evil_family.json`, which could have been downloaded from the internet, contained malicious code, we now lost all of our files.

- This happened because `eval` **evaluates** all parts of the input string as if it were Python code.

- You never need to do this – instead, use the `.json()` method of a response object, or use the `json` library.

<div class="alert alert-danger" markdown="1">

#### Reference Slide

### Using the `json` module

- `json.load(f)` loads a JSON file from a file object.

- `json.loads(f)` loads a JSON file from a **s**tring.

In [ ]:
json.loads(evil_family_str)

- Since `util.err()` is not a string in JSON (there are no quotes around it), `json.loads` is not able to parse it as a JSON object.

- This "safety check" is intentional.

<div class="alert alert-danger" markdown="1">

#### Reference Slide

### Key takeaways

- Never trust data from an unfamiliar site.

- **Never** use `eval` on "raw" data that you didn't create!

- The JSON data format needs to be **parsed**, not evaluated as a dictionary. It was designed with safety in mind!

### Aside: `pd.read_json`

- `pandas` also has a built-in `read_json` function.

In [ ]:
with open('data/family.json', 'r') as f:
    family_df = pd.read_json(f)
family_df

- It only makes sense to use it, though, when you have a JSON file that has some sort of tabular structure. Our family tree example does not.

### API terminology

- A URL, or uniform resource locator, describes the location of a website or resource.

- API requests are just `GET`/`POST` requests to a specially maintained URL.

- As an example, we'll look at the [Pokémon API](https://pokeapi.co).

- All requests are made to:

```
        https://pokeapi.co/api/v2/{endpoint}/{name}
```

- For example, to learn about Pikachu, we use the `pokemon` **endpoint** with name `pikachu`.

        https://pokeapi.co/api/v2/pokemon/pikachu

- Or, to learn about all water Pokemon, we use the `type` endpoint with name `water`.

        https://pokeapi.co/api/v2/pokemon/pikachu

### API requests

- To illustrate, let's make a `GET` request to learn more about Pikachu.

In [ ]:
def request_pokemon(name):
    url = f'https://pokeapi.co/api/v2/pokemon/{name}'
    return requests.get(url)
res = request_pokemon('pikachu')
res

- Remember, the 200 status code is good! Let's take a look at the **content**:

In [ ]:
res.content[:1000]

### Working with JSON objects

- The response we got back looks like JSON. We can extract the JSON from this request with the `json` method (or by passing `res.text` to `json.loads`).

In [ ]:
pikachu = res.json()
pikachu

In [ ]:
pikachu.keys()

In [ ]:
pikachu['weight']

In [ ]:
pikachu['abilities'][1]['ability']['name']

### Invalid `GET` requests

- Let's try a `GET` request for `'wolverine'`.

In [ ]:
request_pokemon('wolverine')

- We receive a 404 error, since there is no Pokemon named `'wolverine'`!

### More on APIs

- We accessed the Pokémon API by making requests. But, some APIs exist as Python _wrappers_, which allow you to make requests by calling Python functions.<br><small>`request_pokemon` is essentially a wrapper for (a small part of) the Pokémon API.</small>

- Some APIs will require you to create an API key, and send that key as part of your request.<br><small>See Homework 5 for an example!</small>

- Many of the APIs you'll use are "REST" APIs. Learn more about RESTful APIs [here](https://en.wikipedia.org/wiki/REST#Architectural_constraints).<br><small>REST stands for "Representational State Transfer." One of the key properties of a RESTful API is that servers don't store any information about previous requests, or who is making them.

## Generalized table manipulation

---

### Representations of tabular data

- In this class, we've worked with DataFrames in `pandas`.

- When we say `pandas` DataFrame, we're talking about the `pandas` API for its DataFrame objects.

- When we say "DataFrame", we're referring to a general way to represent data.<br><small>DataFrames organize data into rows and columns, with labels for both rows and columns.</small>

- There many other ways to work with data tables, each of which have their pros and cons.<br><small>Some examples include R data frames, SQL databases, spreadsheets in Google Sheets/Excel, or even matrices from linear algebra.</small>

### Alternatives

- Below, we discuss pros and cons of (some of) the ways we can work with tabular data.

| Platform               | **Pros ✅**                                          | **Cons ❌**                                            |
|------------------------|--------------------------------------------------|----------------------------------------------------|
| **`pandas` DataFrames**   | Works well with the Python ecosystem (for visualization, machien learning, domain-specifc purposes, etc.), extremely flexible, reproducible steps  | Steep learning curve (need to know Python too) and messy code, easy to make destructive in-place modifications, no persistence (everything starts from a `CSV` file)  |
| **R data frames**        | Designed specifically for data science so statistics and visualizations are easy; reproducible steps | R isn't as general-purpose as Python, no persistence  |
| **SQL**                | Scalable, good for maintaining many large, important datasets with many concurrent users | Requires lots of infrastructure, advanced operations can be challenging  |
| **Spreadsheets**        | Widespread use, very easy to get started, easy for sharing | Steps aren't reproducible, advanced operations can be challenging |

- A common workflow is to load a subset of data in from a database system into `pandas`, then do further cleaning and visualization.

- Another is to load and clean data in `pandas`, then store it in a database system for others to use.

### Relational algebra

- **Relational algebra** captures common data operations between many data table systems.<br><small>We won't test you on relational algebra syntax, but if you'd like to learn more, take EECS 484: Database Management Systems.</small>

- For example, the following expression describes a calculation in relational algebra:

    $$\pi_{\text{int_rate}}(\sigma_{\text{state} = \text{"MI"}}(\text{loans}))$$

    <br><small>$\pi$ stands for "**p**roject," i.e. selecting columns $\sigma$ stands for "**s**elect," i.e. selecting rows.

- In `pandas`, we'd implement this expression as follows:

```python
        loans.loc[loans['state'] == 'MI', 'int_rate']
```

- How would we implement it in SQL? Or a spreadsheet?

### Comparing `pandas`, SQL, and Google Sheets

- To show you how the tabular manipulation skills you've learned in `pandas` generalize to other systems, we will answer a few questions in all three of the above platforms.

- First, we'll work through our analysis in `pandas`, and then in SQL, and finally, in a Google Sheets document.

### Overview: Top 200 streams

- Our dataset contains the number of streams for the top 200 songs on Spotify, for the week leading up to September 19th.<br><small>We downloaded it from [**here**](https://charts.spotify.com/charts/view/regional-global-weekly/latest). This is the most recent week it let us download.</small>

- Let's load it in as a `pandas` DataFrame.

In [ ]:
charts = pd.read_csv('data/regional-global-weekly-2024-09-19.csv') 
charts

In [ ]:
charts.columns 

- As an aside, we can play Spotify songs directly in our notebooks!

In [ ]:
def show_spotify(uri):
    code = uri[uri.rfind(':')+1:]
    src = f"https://open.spotify.com/embed/track/{code}"
    width = 400
    height = 75
    display(IFrame(src, width, height))

In [ ]:
my_uri = charts.loc[charts['track_name'] == 'Yellow', 'uri'].iloc[0] 
my_uri

In [ ]:
show_spotify(my_uri)

<small><small style="color:#4c9eff">Complete using `pandas`.</small></small><br>
**Task 1**: Find the total number of streams of songs by Sabrina Carpenter.

In [ ]:
task_1 = charts.loc[charts['artist_names'] == 'Sabrina Carpenter', 'streams'].sum()
task_1

<small><small style="color:#4c9eff">Complete using `pandas`.</small></small><br>
**Task 2**: Find the total number of streams per artist, sorted by number of streams in descending order. Only show the top 5 artists.

In [ ]:
task_2 = (
    charts
    .groupby('artist_names')
    ['streams']
    .sum()
    .sort_values(ascending=False)
    .head(5)
)
task_2

<small><small style="color:#4c9eff">Complete using `pandas`.</small></small><br>
**Task 3**: Find the artist with the lowest average number of streams, among artists with at least 5 songs in the Top 200.

In [ ]:
task_3 = (
    charts
    .groupby('artist_names')
    .filter(lambda df: df.shape[0] >= 5)
    .groupby('artist_names')
    ['streams']
    .mean()
    .sort_values()
    .head(1)
)
task_3

<small><small style="color:#4c9eff">Complete using `pandas`.</small></small><br>
**Task 4**: Find the number of songs with a higher ranking this week than last week.

In [ ]:
task_4 = charts[charts['rank'] > charts['previous_rank']].shape[0]
task_4

<small><small style="color:#4c9eff">Complete using `pandas`.</small></small><br>
**Task 5**: `charts_old` contains the Top 200 songs in the previous week.

Find the song with the largest increase in streams between last week and this week, among songs that were in the Top 200 in both weeks.

In [ ]:
charts_old = pd.read_csv('data/regional-global-weekly-2024-09-12.csv')
charts_old.head()

In [ ]:
with_old = (
    charts[['uri', 'track_name', 'artist_names', 'streams']]
    .merge(charts_old[['uri', 'streams',]], on='uri', suffixes=('_new', '_old'))
)
task_5 = (
    with_old
    .assign(change=with_old['streams_new'] - with_old['streams_old'])
    .sort_values('change', ascending=False)
    [['track_name', 'artist_names', 'change']]
    .head(1)
)
task_5

<small><small style="color:#4c9eff">Complete using `pandas`.</small></small><br>
**Task 6**: Find the 4 songs with the most artists.

In [ ]:
task_6 = (
    charts
    .assign(num_artists=charts['artist_names'].str.count(', '))
    .sort_values('num_artists', ascending=False)
    .head(4)
)
task_6

## SQL

---

### Overview: SQL

- SQL stands for "Structured Query Language". It is **the** standard language for database manipulation.<br><small>Each database system – for instance, MySQL, SQLite, DuckDB, or PostgreSQL – has its own slightly different version of SQL with special features to that system.</small> 

- SQL is a **declarative** language.<br><small>In SQL, you just describe _what_ you want calculated, not _how_ you want it calculated. It's the database engine's job to figure out _how_ to process your **query**.</small>

```sql
        SELECT artist_names, SUM(streams) AS total_streams FROM charts
        GROUP BY artist_names
        ORDER BY total_streams DESC
        LIMIT 5;
```

<br><center><small>One of the SQL queries we'll write shortly.</small></center>

- We'll work with two SQL database platforms: SQLite and DuckDB.<br><small>SQLite comes pre-installed on most systems. DuckDB is open-source, and integrates with `pandas` really well.</small>

### Example SQL syntax

- All code we write in SQL is referred to as a "query", and all SQL queries follow the same general template:

<center><img src="imgs/sql.png" width=900>

<br>
(<a href="https://learnsql.com/blog/sql-query-basic-elements/">source</a>)

</center>

- [W3Schools](https://www.w3schools.com/sql/sql_examples.asp) has examples of SQL syntax, as does the example above.

### Connecting to a database using `sqlite3`

- `sqlite3` comes pre-installed on most operating systems.

- We'll answer our first few tasks by working with `sqlite3` in the command-line.

- To follow along:
    1. Open your Terminal.
    2. `cd` to the `lec10/data` folder.
    3. Run `sqlite3 spotify.db`.

- These steps will open a `sqlite3` interpreter, connected to the `spotify.db` **database**.<br><small>A database file can contain multiple tables. This one contains two: `charts` and `charts_old`.</small>

<small><small style="color:#ffac3f">Complete using SQL.</small></small><br>
**Task 1**: Find the total number of streams of songs by Sabrina Carpenter.

<center><img src="imgs/task-1.png" width=700></center>

In [ ]:
task_1

<small><small style="color:#ffac3f">Complete using SQL.</small></small><br>
**Task 2**: Find the total number of streams per artist, sorted by number of streams in descending order. Only show the top 5 artists.

<center><img src="imgs/task-2.png" width=700></center>

In [ ]:
task_2

### Aside: DuckDB

- Instead of having to run all of our SQL queries in the Terminal using a `.db` file, we can use DuckDB, which allows us to execute queries in a notebook **using a `pandas` DataFrame**!

- To use DuckDB, `pip install` it:

In [ ]:
!pip install duckdb

In [ ]:
import duckdb

- Then, use the `run_sql` function, defined below, to execute SQL queries using the DataFrames in our notebook as SQL tables.

In [ ]:
def run_sql(query_str, as_df=False):
    out = duckdb.query(query_str)
    if as_df:
        return out.to_df()
    else:
        return out

In [ ]:
run_sql('''
SELECT artist_names, SUM(streams) AS total_streams FROM charts
GROUP BY artist_names
ORDER BY total_streams DESC
LIMIT 5;
''')

- We can even ask for the output back as a DataFrame!<br><small>This means that you can combine both SQL operations and `pandas` operations.</small>

In [ ]:
run_sql('''
SELECT artist_names, SUM(streams) AS total_streams FROM charts
GROUP BY artist_names
ORDER BY total_streams DESC
LIMIT 5;
''', as_df=True)

<small><small style="color:#ffac3f">Complete using SQL.</small></small><br>
**Task 3**: Find the artist with the lowest average number of streams, among artists with at least 5 songs in the Top 200.

In [ ]:
run_sql('''
SELECT artist_names, AVG(streams) as avg_streams FROM charts
GROUP BY artist_names
HAVING COUNT(*) >= 5
ORDER BY avg_streams
LIMIT 1;
''')

In [ ]:
task_3

<small><small style="color:#ffac3f">Complete using SQL.</small></small><br>
**Task 4**: Find the number of songs with a higher ranking this week than last week.

In [ ]:
run_sql('''
SELECT COUNT(*) as num_songs FROM (
    SELECT * FROM charts
    WHERE rank > previous_rank
)
''')

In [ ]:
task_4

<small><small style="color:#ffac3f">Complete using SQL.</small></small><br>
**Task 5**: `charts_old` contains the Top 200 songs in the previous week.

Find the song with the largest increase in streams between last week and this week, among songs that were in the Top 200 in both weeks.

In [ ]:
run_sql('''
SELECT track_name, artist_names, (new_streams - old_streams) AS change
FROM (
    SELECT charts.uri, 
           charts.track_name, 
           charts.artist_names, 
           charts.streams AS new_streams, 
           charts_old.uri, 
           charts_old.streams AS old_streams
    FROM charts
    INNER JOIN charts_old ON charts.uri = charts_old.uri
) AS merged
ORDER BY change DESC
LIMIT 1;
''', as_df=True)

In [ ]:
task_5

<small><small style="color:#ffac3f">Complete using SQL.</small></small><br>
**Task 6**: Find the 4 songs with the most artists.<br>
<small>Fun fact: the syntax used in our solution doesn't exist in SQLite, but **does** exist in DuckDB.</small>

In [ ]:
run_sql('''
SELECT track_name, artist_names, array_length(str_split(artist_names, ', ')) AS num_artists
FROM charts
ORDER BY num_artists DESC
LIMIT 4;
''', as_df=True)

In [ ]:
task_6

## Google Sheets

---

### Overview: Google Sheets

- While we're big fans of writing code in this class, in the business world, spreadsheets are (still) the most common tool for tabular data manipulation.

- Spreadsheets are great for showing information directly to someone else, and are easy to share.

- Microsoft Excel is widely popular, but Google Sheets is increasingly common as well, and you've likely used it before.

- Follow along by opening [**this Google Sheet**](https://docs.google.com/spreadsheets/d/1OFWqms0UN-9Lr5HxwPdZsgpPlLir8fy67dN0PD0beug/edit?usp=sharing) and going to File > Make a copy.

<small><small style="color:#b96fff">Complete using Google Sheets.</small></small><br>
**Task 1**: Find the total number of streams of songs by Sabrina Carpenter.

Relevant functions: `FILTER`, `SUMIF`.

In [ ]:
task_1

<small><small style="color:#b96fff">Complete using Google Sheets.</small></small><br>
**Task 2**: Find the total number of streams per artist, sorted by number of streams in descending order. Only show the top 5 artists.

Relevant functions: `UNIQUE`, `SUMIF`.

In [ ]:
task_2

<small><small style="color:#b96fff">Complete using Google Sheets.</small></small><br>
**Task 3**: Find the artist with the lowest average number of streams, among artists with at least 5 songs in the Top 200.

Relevant functions: `AVERAGEIF`.

In [ ]:
task_3

<small><small style="color:#b96fff">Complete using Google Sheets.</small></small><br>
**Task 4**: Find the number of songs with a higher ranking this week than last week.

Relevant functions: `IF`.

In [ ]:
task_4

<small><small style="color:#b96fff">Complete using Google Sheets.</small></small><br>
**Task 5**: `charts_old` contains the Top 200 songs in the previous week.

Find the song with the largest increase in streams between last week and this week, among songs that were in the Top 200 in both weeks.

Relevant functions: `FILTER`.

In [ ]:
task_5

<small><small style="color:#b96fff">Complete using Google Sheets.</small></small><br>
**Task 6**: Find the 4 songs with the most artists.

Relevant functions: `SPLIT`, `LEN`, `SUBSTITUTE`.

In [ ]:
task_6

### Key takeaways

- The DataFrame manipulation techniques we've learned about over the past month generalize to other systems that you might be exposed to.

- In 20 years, `pandas` may not exist, but grouping, pivoting, querying, etc. are all **concepts** that still will be useful.

### What's next?

- Next week, we'll learn how to extract information from messy, text data.

- Then, after the Midterm Exam, we'll switch our focus to machine learning!